This notebook focuses on examples inspired by Kurose & Ross's Computer Networking: A Top-Down Approach.

As always, we start by loading ns-3.

In [1]:
import sys
sys.path.append("./ns-3-dev/build/bindings/python")
sys.path.append("./ns-3-dev/build/lib")
from ns import ns

[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { $.cling-module-148.__inits.0, _GLOBAL__sub_I_cling_module_148, __orc_init_func.cling-module-148, _ZN3ns3L16g_timeInitHelperE, __cxx_global_var_initcling_module_148_ }) }
[runStaticInitializersOnce]: Failed to materialize symbols: { (main, { __orc_init_func.cling-module-148 }) }


Now we can start building our examples. Remember to always destroy the simulator state before rerunning cells.

In [2]:
ns.Simulator.Destroy()

# First example: network stack (based on 6th Edition, Chapter 1)

Computer networks are like onions. Layers! 

Onions have layers, and computer network have layers (yes, this is a Shrek reference). 

The widely used Internet, or TCP/IP, stack is composed of five layers.

```mermaid
flowchart 

subgraph NS[Network Stacks]
  direction LR

subgraph stackA[Node 1]
  direction TB
  A[Application]-->|sockets|B[Transport];
  B-->C[Network]; 
  C-->D[Link];
  D-->E[Physical];
end
subgraph stackB[Node 2]
  direction BT
  J[Physical]-->I[Link];
  I-->H[Network];
  H-->G[Transport]; 
  G-->|sockets|F[Application];
end
stackA-->|physical media|stackB;
end
```

The application layer is where applications such as web browsers, games, office suites, banking  servicess, etc generate and consume payloads that will be transported by the network.

The transport layer acts as an intermediate between the application and the underlying network, implementing basic services/protocolss or more complex ones, that try to offer more guarantees to the application, such as error checking and automatic retransmission, automatic ordering, etc. Packets that arrive or leave the transsport layer have source and destination ports, that identify which process at the sender and the receiver are trying to communicate across a network route composed of many links.

The network layer implements logical routes. Theses routes are based on forwarding packets from one network node to another, connected by a physical link. These packet forwardings are also known as hops, which are limited via a counter called Time-To-Live (TTL). When TTL reaches 0, the packet is discarded, and the original sender retransmits it (in case it is using a reliable transport protocol).

The link layer and physical layer are typically combined in a network interface, or network device. But you probably know them by their popular names "network card" and "modem". The link layer is also commonly refered to as the Medium-Access Control (MAC) layer, becauses it controls the physical layer access to the physical media.

The physical layer of both sender (Node 1) and receiver (Node 2) are then physically connected by a physical medium, such as air (e.g. morse code, wifi, lifi, 2G/3G/4G/5G/6G, and even smoke signal), water (e.g. UAN), metals such as copper (e.g. telegraph, landline, ethernet, thunderbolt, USB), glass (e.g. optical fiber), plastic (e.g. TOSLINK).  

The physical medium, or part of its capacity to transmit data, is typically referred to as a channel. The channel may have different capacitiess depending on the medium properties. Optical fiber, for example, is far less susceptible to interference, allowing for much higher capacity than copper, especially in long links, such as the submarine cables that connect countries and continents to a single global internet.

Enough with the chit chat, let us see a real example of a network stack, built step by step.

In [ ]:
def simu_stack(withApps=False):
    ns.Simulator.Destroy()

    if withApps:
        pass

simu_stack()

There we have it. We have our network composed of two nodes, with their respective stacks, connected by a channel.

But we can't tell for sure if it is working or not. We need applications to inject traffic for that.

In [ ]:
simu_stack(withApps=True)

# Second example: network packets (based on 6th Edition, Chapter 2, R19)

# Third example: network flow (based on 6th Edition, Chapter 1, R19)

# Fourth example: network delay (based on 6th Edition, Chapter 1, R18 and P7)


# Fifth example: network bottlenecks (based on 6th Edition, Chapter 1, R19)

Computer networks are formed by a series of links connecting devices physically via a medium.

Those links can be interconnected logically via routes, in which nodes receive packets not meant for them, and forward them to other links, until the packet reaches its final destination or is discarded.

You may have experienced a side-effect of those logical routes, for example when you subscribe to a 1Gbps internet plan, but has to wait for downloads to complete, because downloads are still at meager 10s-100s of Mbps.

This typically happen due to bottlenecks in the route, which are limited by the lowest capacity link. This is why Speedtest.net and other similar services typically pick a nearby server to measure your internet speed, reducing the number of link in the route, which translates to a smaller chance of being bottlenecked by a link other than your ISP network and the remote server.

Let's see this bottleneck in action by building a network composed of four nodes, connected by links of different capacities.

```mermaid
flowchart LR
A[A];
B[B];
C[C];
D[D];
A-->|500kbps|B;
B-->|2Mbps|C; 
C-->|1Mbps|D;
```

We want to send a big payload from A to D, for example 4MB. We also want to find our how long it takes to send this file.
The simulation for this can be modelled as follows:

In [3]:
def simu_bottleneck(link_thrs=["500kbps", "2Mbps", "1Mbps"]):
    ns.Simulator.Destroy()
    topology = ns.NodeContainer(4) # create the nodes A, B, C, D
    for i in range(3):
        # Create a node container for each pair
        cn = ns.NodeContainer()
        cn.Add(topology.Get(i))
        cn.Add(topology.Get(i+1))
               
        # Install a point-to-point link between them
        csma = ns.CsmaHelper()
        csma.SetChannelAttribute("DataRate", ns.StringValue(link_thrs[i]))
        devices = csma.Install(cn)

        # Install the IP stack
        stack = ns.InternetStackHelper()
        stack.Install(cn)

        # Set Ipv4 Addresses
        address = ns.Ipv4AddressHelper()
        address.SetBase(ns.Ipv4Address(f"10.1.{i}.0"), ns.Ipv4Mask("255.255.255.0"))
        interfaces = address.Assign(devices)

    # Populate routing tables
    ns.Ipv4GlobalRoutingHelper.PopulateRoutingTables()

    # Now that we have our topology, we need to setup our application to inject 
    # 4MB of traffic from A to D, and a sink application at D to receive this data
    sourceApplications = ns.ApplicationContainer()
    sinkApplications = ns.ApplicationContainer()

    # Install traffic source at node A
    ipv4 = topology.Get(3).GetObject[ns.Ipv4]()
    address = ipv4.GetAddress(1, 0).GetLocal()
    sourceHelper = ns.OnOffHelper("ns3::UdpSocketFactory", ns.InetSocketAddress(address, 80).ConvertTo())
    # the application should transmit with a high data rate to saturate the links
    sourceHelper.SetAttribute("DataRate", ns.DataRateValue(ns.DataRate("10Mbps"))) 
    sourceHelper.SetAttribute("MaxBytes", ns.UintegerValue(4*1024*1024))  # transmit a total of 4 MB
    sourceHelper.SetAttribute("PacketSize", ns.UintegerValue(1024))  # in packets of 1 KB
    sourceHelper.SetAttribute("OffTime", ns.StringValue("ns3::ConstantRandomVariable[Constant=0.0]")) # always transmit
    sourceApplications.Add(sourceHelper.Install(topology.Get(0)))
    sourceApplications.Start(ns.Seconds(1.0))

    # Create sink sink at node D
    packetSinkHelper = ns.PacketSinkHelper("ns3::UdpSocketFactory", ns.InetSocketAddress(ns.Ipv4Address.GetAny(), 80).ConvertTo())
    sinkApplications.Add(packetSinkHelper.Install(topology.Get(3)))
    sinkApplications.Start(ns.Seconds(1))

    flowmon_helper = ns.FlowMonitorHelper()
    monitor = flowmon_helper.InstallAll()

    # Start simulator
    ns.Simulator.Stop(ns.Minutes(1))
    ns.Simulator.Run()

    # Print results
    print("flows", len(list(monitor.GetFlowStats())))
    flow_id, flow_stats = list(monitor.GetFlowStats())[0]
    txBytes = flow_stats.txBytes
    rxBytes = flow_stats.rxBytes
    txTime = (flow_stats.timeLastRxPacket-flow_stats.timeFirstRxPacket).GetSeconds()
    thr = rxBytes*8/txTime
    thrUnits = ["bps", "kbps", "Mbps", "Gbps", "Tbps"]
    while thr > 1024:
        thr /= 1024
        thrUnits.pop(0)
    print("Total bytes sent:", txBytes)
    print("Total bytes received:", rxBytes)
    print("Total transmission time:", txTime)
    print("Throughput:", thr, thrUnits[0])

simu_bottleneck()

flows 1
Total bytes sent: 4308992
Total bytes received: 810040
Total transmission time: 13.303388
Throughput: 475.7011898021767 kbps


The simulation results from above tell us that A transmitted ~4.3MB in approximately 13 seconds (the 0.3 MB come from packet header and/or trailer overhead, added at each network layer).

Only 810 kilobytes were received by D, at an average throughput of 475 kbps.

Note that the observed throughput is very close to the lowest capacity link between A and B, which is what we expected.

To double-check we didn't make any mistake, we can repeat our experiment lowering the capacity of the link between B and C, for example, from 2 Mbps to 100 kbps.

```mermaid
flowchart LR
A[A];
B[B];
C[C];
D[D];
A-->|500kbps|B;
B-->|100kbps|C; 
C-->|1Mbps|D;
```

In [4]:
simu_bottleneck(["500kbps", "100kbps", "1Mbps"])

flows 1
Total bytes sent: 4308992
Total bytes received: 269312
Total transmission time: 22.071774
Throughput: 95.32536895312538 kbps


The transmission time increased, the number of received bytes decreased and the throughput was also reduced.

Note that the throughput is close to the lowest capacity link between B and C, as predicted.

Now let us see what happens if our internet provider (e.g. A-B link) is upgraded, and the core network links are of higher capacity.

In [6]:
simu_bottleneck(["10Mbps", "1Gbps", "1Gbps"])

flows 1
Total bytes sent: 4308992
Total bytes received: 4205896
Total transmission time: 3.457587552
Throughput: 9.280586379611725 Mbps


Yet again, the achievable throughput is bound by the link with the lowest capacity.

Remember this is the case before calling to your ISP and cursing the poor attendants that do not control the network infrastructure.

# Sixth example: sockets (based on 6th Edition, Chapter 2, R19)
